<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Le Havre 

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Le_Havre_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Le Havre - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1

In [7]:
class HavreSpider(scrapy.Spider) :
    # Name of spider
    name = 'Le_Havre'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4AuCFw5QGwAIB0gIkM2M4NzY3NjEtY2YyOS00NDcyLTk2ZDktOGQ5MGIwODE5OTVi2AIG4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=le+havre&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=687b5730db290148&ss_raw=le+havre&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(HavreSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
le_havre = pd.read_json('results_hotels/Le_Havre_multipage.json')

In [11]:
len(le_havre)

125

In [12]:
le_havre.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,"Perret, Le Havre",https://www.booking.com/hotel/fr/la-baraka.fr....,Hôtel Le Marignan - Le Havre Centre Gare,"8,0","Situé au Havre, à 2,1 km de la plage, L'Hôtel ..."
1,Le Havre,https://www.booking.com/hotel/fr/campanile-le-...,Holiday Inn Express - Le Havre Centre,"8,2","L’Holiday Inn Express - Le Havre Centre, situé..."
2,Le Havre,https://www.booking.com/hotel/fr/premiere-clas...,Premiere Classe Le Havre Centre-LES DOCKS,"7,6","Situé au Havre, à proximité des autoroutes A13..."
3,Le Havre,https://www.booking.com/hotel/fr/nomad-le-havr...,Nomad Hotel le Havre,"8,5",Le Nomad Hotel Le Havre propose des chambres c...
4,"Perret, Le Havre",https://www.booking.com/hotel/fr/pasino.fr.htm...,Hotel Spa Le Pasino,"8,4","Situé au cœur du Havre, l'Hotel Spa le Pasino ..."
5,Le Havre,https://www.booking.com/hotel/fr/un-petit-coco...,Un petit COCON,"7,0",Hébergement géré par un particulier
6,"Perret, Le Havre",https://www.booking.com/hotel/fr/carmin.fr.htm...,Hôtel Carmin,"8,5",L'Hôtel Carmin est situé dans le centre-ville ...
7,"Perret, Le Havre",https://www.booking.com/hotel/fr/le-richelieu-...,Le Richelieu,"7,2",Le Citotel Le Richelieu est situé entre la mai...
8,Le Havre,https://www.booking.com/hotel/fr/hotelterminus...,Best Western Plus Le Havre Centre Gare,"8,6","Situé dans le centre du Havre, à seulement 10 ..."
9,"Perret, Le Havre",https://www.booking.com/hotel/fr/vent-d-ouest-...,Hôtel Spa Vent d'Ouest,"8,1","L’Hôtel Spa Vent d’Ouest se situe au Havre, à ..."


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in le_havre['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
le_havre['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in le_havre['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
le_havre['latitude'] = lat
le_havre['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in le_havre['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
le_havre['hotel_description_long'] = result_long_description

In [20]:
le_havre.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Perret, Le Havre",https://www.booking.com/hotel/fr/la-baraka.fr....,Hôtel Le Marignan - Le Havre Centre Gare,"8,0","Situé au Havre, à 2,1 km de la plage, L'Hôtel ...","49.49227908,0.12310803",49.49227908,0.12310803,"\nSitué au Havre, à 2,1 km de la plage, L'Hôte..."
1,Le Havre,https://www.booking.com/hotel/fr/campanile-le-...,Holiday Inn Express - Le Havre Centre,"8,2","L’Holiday Inn Express - Le Havre Centre, situé...","49.49823800,0.12883700",49.49823800,0.12883700,"\nL’Holiday Inn Express - Le Havre Centre, sit..."
2,Le Havre,https://www.booking.com/hotel/fr/premiere-clas...,Premiere Classe Le Havre Centre-LES DOCKS,"7,6","Situé au Havre, à proximité des autoroutes A13...","49.49641693,0.15151234",49.49641693,0.15151234,"\nSitué au Havre, à proximité des autoroutes A..."
3,Le Havre,https://www.booking.com/hotel/fr/nomad-le-havr...,Nomad Hotel le Havre,"8,5",Le Nomad Hotel Le Havre propose des chambres c...,"49.49331760,0.12530688",49.49331760,0.12530688,\nVous pouvez bénéficier d'une réduction Geniu...
4,"Perret, Le Havre",https://www.booking.com/hotel/fr/pasino.fr.htm...,Hotel Spa Le Pasino,"8,4","Situé au cœur du Havre, l'Hotel Spa le Pasino ...","49.49117449,0.11193663",49.49117449,0.11193663,\nVous pouvez bénéficier d'une réduction Geniu...
5,Le Havre,https://www.booking.com/hotel/fr/un-petit-coco...,Un petit COCON,"7,0",Hébergement géré par un particulier,"49.50089060,0.14118180",49.50089060,0.14118180,\nL'hébergement Un petit COCON est situé au Ha...
6,"Perret, Le Havre",https://www.booking.com/hotel/fr/carmin.fr.htm...,Hôtel Carmin,"8,5",L'Hôtel Carmin est situé dans le centre-ville ...,"49.49493852,0.10553956",49.49493852,0.10553956,\nVous pouvez bénéficier d'une réduction Geniu...
7,"Perret, Le Havre",https://www.booking.com/hotel/fr/le-richelieu-...,Le Richelieu,"7,2",Le Citotel Le Richelieu est situé entre la mai...,"49.48881105,0.10760218",49.48881105,0.10760218,\nCet établissement est à 13 minutes à pied de...
8,Le Havre,https://www.booking.com/hotel/fr/hotelterminus...,Best Western Plus Le Havre Centre Gare,"8,6","Situé dans le centre du Havre, à seulement 10 ...","49.49334357,0.12431771",49.49334357,0.12431771,\nVous pouvez bénéficier d'une réduction Geniu...
9,"Perret, Le Havre",https://www.booking.com/hotel/fr/vent-d-ouest-...,Hôtel Spa Vent d'Ouest,"8,1","L’Hôtel Spa Vent d’Ouest se situe au Havre, à ...","49.49141928,0.10070756",49.49141928,0.10070756,\nCet établissement est à 7 minutes à pied de ...


In [21]:
le_havre['hotel_description_long'] = le_havre['hotel_description_long'].replace('\n','', regex=True)

In [22]:
le_havre.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Perret, Le Havre",https://www.booking.com/hotel/fr/la-baraka.fr....,Hôtel Le Marignan - Le Havre Centre Gare,"8,0","Situé au Havre, à 2,1 km de la plage, L'Hôtel ...","49.49227908,0.12310803",49.49227908,0.12310803,"Situé au Havre, à 2,1 km de la plage, L'Hôtel ..."
1,Le Havre,https://www.booking.com/hotel/fr/campanile-le-...,Holiday Inn Express - Le Havre Centre,"8,2","L’Holiday Inn Express - Le Havre Centre, situé...","49.49823800,0.12883700",49.49823800,0.12883700,"L’Holiday Inn Express - Le Havre Centre, situé..."
2,Le Havre,https://www.booking.com/hotel/fr/premiere-clas...,Premiere Classe Le Havre Centre-LES DOCKS,"7,6","Situé au Havre, à proximité des autoroutes A13...","49.49641693,0.15151234",49.49641693,0.15151234,"Situé au Havre, à proximité des autoroutes A13..."
3,Le Havre,https://www.booking.com/hotel/fr/nomad-le-havr...,Nomad Hotel le Havre,"8,5",Le Nomad Hotel Le Havre propose des chambres c...,"49.49331760,0.12530688",49.49331760,0.12530688,Vous pouvez bénéficier d'une réduction Genius ...
4,"Perret, Le Havre",https://www.booking.com/hotel/fr/pasino.fr.htm...,Hotel Spa Le Pasino,"8,4","Situé au cœur du Havre, l'Hotel Spa le Pasino ...","49.49117449,0.11193663",49.49117449,0.11193663,Vous pouvez bénéficier d'une réduction Genius ...


In [23]:
le_havre['hotel_description_long'][55]

"Cet établissement est à 18 minutes à pied de la plage. L'ibis Le Havre Centre est situé dans le centre-ville du Havre, à proximité de la gare SNCF, du port ferry et du centre du commerce international. Une connexion Wi-Fi gratuite est disponible.Les chambres disposent d'une salle de bains privative, d'une télévision par satellite et de matériel de repassage.Un petit-déjeuner buffet, composé de plats sucrés et salés, comme des œufs, de la salade de fruits, des yaourts et des jus, est servi chaque matin. Des viennoiseries cuites sur place, des madeleines fraîches ainsi qu'une boisson chaude et un fruit à emporter sont aussi proposés. Vous pourrez par ailleurs déguster un petit-déjeuner plus léger à partir de 4h00.Depuis l'ibis Le Havre Centre, vous rejoindrez facilement les autoroutes A29 et A131. Honfleur se trouve à 23 km, tandis que Deauville vous attend à 40 km. Enfin, vous pourrez profiter d'un espace de stationnement et d'une salle de réunion sur place. "

In [24]:
# Clean location column
le_havre['location'] = ["Le Havre" if value!="Le Havre" else value for value in le_havre["location"]]

In [26]:
le_havre.head(25)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Le Havre,https://www.booking.com/hotel/fr/la-baraka.fr....,Hôtel Le Marignan - Le Havre Centre Gare,"8,0","Situé au Havre, à 2,1 km de la plage, L'Hôtel ...","49.49227908,0.12310803",49.49227908,0.12310803,"Situé au Havre, à 2,1 km de la plage, L'Hôtel ..."
1,Le Havre,https://www.booking.com/hotel/fr/campanile-le-...,Holiday Inn Express - Le Havre Centre,"8,2","L’Holiday Inn Express - Le Havre Centre, situé...","49.49823800,0.12883700",49.49823800,0.12883700,"L’Holiday Inn Express - Le Havre Centre, situé..."
2,Le Havre,https://www.booking.com/hotel/fr/premiere-clas...,Premiere Classe Le Havre Centre-LES DOCKS,"7,6","Situé au Havre, à proximité des autoroutes A13...","49.49641693,0.15151234",49.49641693,0.15151234,"Situé au Havre, à proximité des autoroutes A13..."
3,Le Havre,https://www.booking.com/hotel/fr/nomad-le-havr...,Nomad Hotel le Havre,"8,5",Le Nomad Hotel Le Havre propose des chambres c...,"49.49331760,0.12530688",49.49331760,0.12530688,Vous pouvez bénéficier d'une réduction Genius ...
4,Le Havre,https://www.booking.com/hotel/fr/pasino.fr.htm...,Hotel Spa Le Pasino,"8,4","Situé au cœur du Havre, l'Hotel Spa le Pasino ...","49.49117449,0.11193663",49.49117449,0.11193663,Vous pouvez bénéficier d'une réduction Genius ...
5,Le Havre,https://www.booking.com/hotel/fr/un-petit-coco...,Un petit COCON,"7,0",Hébergement géré par un particulier,"49.50089060,0.14118180",49.50089060,0.14118180,L'hébergement Un petit COCON est situé au Havr...
6,Le Havre,https://www.booking.com/hotel/fr/carmin.fr.htm...,Hôtel Carmin,"8,5",L'Hôtel Carmin est situé dans le centre-ville ...,"49.49493852,0.10553956",49.49493852,0.10553956,Vous pouvez bénéficier d'une réduction Genius ...
7,Le Havre,https://www.booking.com/hotel/fr/le-richelieu-...,Le Richelieu,"7,2",Le Citotel Le Richelieu est situé entre la mai...,"49.48881105,0.10760218",49.48881105,0.10760218,Cet établissement est à 13 minutes à pied de l...
8,Le Havre,https://www.booking.com/hotel/fr/hotelterminus...,Best Western Plus Le Havre Centre Gare,"8,6","Situé dans le centre du Havre, à seulement 10 ...","49.49334357,0.12431771",49.49334357,0.12431771,Vous pouvez bénéficier d'une réduction Genius ...
9,Le Havre,https://www.booking.com/hotel/fr/vent-d-ouest-...,Hôtel Spa Vent d'Ouest,"8,1","L’Hôtel Spa Vent d’Ouest se situe au Havre, à ...","49.49141928,0.10070756",49.49141928,0.10070756,Cet établissement est à 7 minutes à pied de la...


In [27]:
le_havre.to_csv("Le_Havre.csv")